In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# 데이터 불러오기

In [ ]:
cctv = pd.read_csv('서울cctv.csv')

In [ ]:
pop = pd.read_csv('서울인구.csv')

In [ ]:
crime = pd.read_csv('서울범죄.csv',encoding='utf-8')

In [ ]:
cctv.head()

In [ ]:
pop.head()

In [ ]:
# 컬럼명 변경

pop.rename(columns = {'동별(2)':'자치구', '구분별(1)':'국적', '구분별(2)':'성별', '데이터':'인구수'}, inplace=True)

In [ ]:
crime.head()

In [ ]:
# 필요없는 컬럼 삭제

cctv.drop(columns = ['수정 일시'], inplace=True)
pop.drop(columns = ['시점'], inplace=True)

In [ ]:
pop['동별(1)'].unique()

In [ ]:
pop.drop(columns = ['동별(1)'], inplace=True)

In [ ]:
cctv.info()

In [ ]:
pop.info()

In [ ]:
crime.info()

In [ ]:
crime['구분'].unique()

In [ ]:
crime.rename(columns={'구분':'관서'}, inplace=True)

In [ ]:
# crime데이터에 자치구 정보 추가를 위한 경찰서 정보 데이터 불러오기

pol = pd.read_csv('서울경찰서.csv')

In [ ]:
pol.head()

In [ ]:
# 관서명 변경

In [ ]:
pol['관서'] = pol['관서'].str.replace('서울', '')
pol['관서'] = pol['관서'].str.replace('경찰서', '')

pol

In [ ]:
# 자치구 컬럼 생성

In [ ]:
pol['자치구'] = pol['주소'].str.split(' ').str[1]

In [ ]:
pol.drop(columns = ['주소'], inplace=True)

In [ ]:
# crime 데이터와 합치기

In [ ]:
crime = pd.merge(crime, pol, on = '관서')

In [ ]:
crime

In [ ]:
# pivot하여 데이터 정리
crime.drop(columns = ['관서'], inplace=True)

crime = crime.pivot_table(crime, index=['자치구'],
                          columns=['죄종', '발생검거'],
                          aggfunc=[np.sum])
crime

In [ ]:
# Null값 0으로 대치

crime = crime.fillna(0)
crime.isnull().sum()

In [ ]:
# sum, 건수 컬럼명 삭제

crime.columns = crime.columns.droplevel([0, 1])
crime.head()

In [ ]:
# 레벨0, 레벨1의 컬럼명을 합쳐 컬럼명 변경

tmp = [
    crime.columns.get_level_values(0)[n] + " "
    + crime.columns.get_level_values(1)[n]
    for n in range(0, len(crime.columns.get_level_values(0)))
]
crime.columns = tmp
crime.head()

In [ ]:
# 검거율 계산

target = ["강간 검거율", "강간,추행 검거율", "강도 검거율", "살인 검거율", "절도 검거율", "폭력 검거율"]
num = ["강간 검거", "강간,추행 검거", "강도 검거", "살인 검거", "절도 검거", "폭력 검거"]
den = ["강간 발생", "강간,추행 발생", "강도 발생", "살인 발생", "절도 발생", "폭력 발생"]

crime[target] = crime[num].div(crime[den].values)
crime.head()

In [ ]:
# 검거 삭제

crime.drop(columns = ["강간 검거", "강간,추행 검거", "강도 검거", "살인 검거", "절도 검거", "폭력 검거"], inplace=True)

In [ ]:
# 컬럼명 변경

crime.rename(
    columns={"강간 발생" : "강간", "강간,추행 발생" : "강간,추행", "강도 발생" : "강도", "살인 발생" : "살인", "절도 발생" : "절도", "폭력 발생" : "폭력"},
    inplace=True
)

In [ ]:
crime

In [ ]:
col = ["강간", "강간,추행", "강도", "살인", "절도", "폭력"]
col2 = ["강간 검거율", "강간,추행 검거율", "강도 검거율", "살인 검거율", "절도 검거율", "폭력 검거율"]

# 사건 발생 수 최소-최대 정규화
crime_norm = (crime[col]-crime[col].min())/(crime[col].max()-crime[col].min())
crime_norm[col2] = crime[col2]
crime_norm.head()

In [ ]:
# 검거율 DF

crime_rate = crime_norm.drop(columns = ["강간", "강간,추행", "강도", "살인", "절도", "폭력"])

In [ ]:
crime_rate

In [ ]:
# 자치구별 검거율 히트맵

import seaborn as sns
import matplotlib.pyplot as plt
import platform

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system... sorry~~~~') 


plt.figure(figsize=(10, 8))
sns.heatmap(crime_rate, annot=False, cmap="YlGnBu")
plt.show()

In [ ]:
cctv

In [ ]:
cctv = cctv.groupby('자치구').sum()

In [ ]:
# cctv 데이터 주소, 위도, 경도 컬럼 삭제

cctv.drop(columns = ["안심 주소", "위도", "경도"], inplace=True)

In [ ]:
cctv

In [ ]:
# 인구수 데이터 국적, 성별 컬럼 삭제

pop = pop.groupby('자치구').sum()
pop.drop(columns = ["국적", "성별"], inplace=True)
pop

In [ ]:
# cctv 데이터와 인구수 데이터 합치기

seoul = pd.merge(cctv, pop, on='자치구')

In [ ]:
seoul

In [ ]:
fp1 = np.polyfit(seoul['인구수'], seoul['CCTV 수량'], 1)

f1 = np.poly1d(fp1)
fx = np.linspace(100000, 700000, 100)

plt.figure(figsize=(14, 10))
plt.scatter(seoul['인구수'], seoul['CCTV 수량'], s=50)
plt.plot(fx, f1(fx), ls='dashed', lw=3, color='g')
plt.xlabel('인구수')
plt.ylabel('CCTV 수량')
for n in range(25):
    plt.text(seoul['인구수'][n], seoul['CCTV 수량'][n], seoul.index[n], fontsize=10)
plt.grid()
plt.show()

* 인구수가 많을수록 CCTV수량이 많은 것으로 보임

In [ ]:
# 상관계수

seoul.corr()

In [ ]:
# 범죄 데이터와 합치기

seoul_crime = pd.merge(seoul, crime_norm, on='자치구')

In [ ]:
seoul_crime

In [ ]:
y1 = seoul_crime["강도 검거율"]
y2 = seoul_crime["강도"]


plt.plot(seoul_crime.index, y1, color = 'red')
plt.plot(seoul_crime.index, y2, color = 'blue')

plt.show()

In [ ]:
crime_norm.corr()

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(crime_norm.corr(), cmap = "RdBu_r", annot=False)
plt.show()

In [ ]:
sns.pairplot(seoul_crime, vars=["인구수","CCTV 수량","강도 검거율"], kind="reg", height=3)
plt.show()

In [ ]:
sns.pairplot(seoul_crime, vars=["인구수","CCTV 수량","살인 검거율"], kind="reg", height=3)
plt.show()

In [ ]:
sns.pairplot(seoul_crime, vars=["인구수","CCTV 수량","폭력 검거율"], kind="reg", height=3)
plt.show()

In [ ]:
corr = seoul_crime.corr()

In [ ]:
corr

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(corr, cmap = "RdBu_r", annot=False)
plt.show()